# Convert a HDF5 file into a multiframe TIFF

This simple tutorial explains how to convert a mutiframe HDF5 into a Tiff.
FabIO does not support mutiframe TIFF file writing (reading is OK) and provides TiffIO which supports it.

In [1]:
import os
import fabio
from fabio.test.utilstest import UtilsTest
from fabio.TiffIO import TiffIO

In [2]:
filename = UtilsTest.getimage("sample_water0000.h5")
print(f"Just downloaded {filename}")

Just downloaded /tmp/fabio_testdata_kieffer/sample_water0000.h5


In [3]:
fimg = fabio.open(filename)
print(f"This file contains {fimg.nframes} frames of shape {fimg.shape} and weights {os.stat(filename).st_size/1e6:.3f}MB")

This file contains 10 frames of shape (1679, 1475) and weights 99.101MB


In [4]:
#Initialization of the TiffIO file with the first frame, note the mode "w"
dest = filename.replace(".h5", ".tiff")
tif = TiffIO(dest, mode='w')
tif.writeImage(fimg.data)
del  tif

In [5]:
# Complete the Tiff file with all other frames, note the mode "r+"
tif = TiffIO(dest, mode='r+')
for frame_id in range(1, fimg.nframes):
    tif.writeImage(fimg.get_frame(frame_id).data)
print(f"The destination file {dest} contains {tif.getNumberOfImages()} frames  and weights {os.stat(dest).st_size/1e6:.3f}MB")
del tif

The destination file /tmp/fabio_testdata_kieffer/sample_water0000.tiff contains 10 frames  and weights 99.062MB


In [6]:
timg = fabio.open(dest)
print(f"{'frame id':10s}|{'min hdf5':10s}|{'min tiff':10s}|{'max hdf5':10s}|{'max tiff':10s}|{'mean hdf5':10s}|{'mean tiff':10s}|{'std hdf5':10s}|{'std tiff':10s}|{'hdf5==tiff':10s}")
for frame_id in range(fimg.nframes):
    hdata = fimg.get_frame(frame_id).data
    tdata = timg.get_frame(frame_id).data
    print(f"{frame_id:10d}|{hdata.min():10d}|{tdata.min():10d}|{hdata.max():10d}|{tdata.max():10d}|{hdata.mean():10f}|{tdata.mean():10f}|{hdata.std():10f}|{tdata.std():10f}|{(hdata==tdata).all()}")
# The two files have actually the same content

frame id  |min hdf5  |min tiff  |max hdf5  |max tiff  |mean hdf5 |mean tiff |std hdf5  |std tiff  |hdf5==tiff
         0|        -2|        -2|    284956|    284956|  6.854841|  6.854841|181.234427|181.234427|True
         1|        -2|        -2|    284956|    284956|  6.853417|  6.853417|181.238234|181.238234|True
         2|        -2|        -2|    284956|    284956|  6.854140|  6.854140|181.265561|181.265561|True
         3|        -2|        -2|    284956|    284956|  6.844210|  6.844210|181.167266|181.167266|True
         4|        -2|        -2|    284956|    284956|  6.847996|  6.847996|181.168757|181.168757|True
         5|        -2|        -2|    284956|    284956|  6.853962|  6.853962|181.265936|181.265936|True
         6|        -2|        -2|    284956|    284956|  6.855220|  6.855220|181.327199|181.327199|True
         7|        -2|        -2|    284956|    284956|  6.851232|  6.851232|181.309509|181.309509|True
         8|        -2|        -2|    284956|    284956|  6

## Conclusion

This simple tutorial explains how to perform a file conversion towards multiframe TIFF. 

Note the importance of the **opening mode** in TiffIO!
